<a href="https://colab.research.google.com/github/raduga256/fine-tuning-classifiers/blob/main/RandomFClassifier_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

my aim was not to build a robust classifier, rather I wanted to show the practicality of optimizing a classifier for sensitivity. In figure A below, the goal is to move the decision threshold to the left. This minimizes false negatives, which are especially troublesome in the dataset chosen for this post. It contains features from images of 357 benign and 212 malignant breast biopsies. A false negative sample equates to missing a diagnosis of a malignant tumor. 

With scikit-learn, tuning a classifier for recall can be achieved in (at least) two main steps.
Using GridSearchCV to tune your model by searching for the best hyperparameters and keeping the classifier with the highest recall score.
Adjust the decision threshold using the precision-recall curve and the roc curve, which is a more involved method..

In [24]:
# Start by loading the necessary libraries and the data.
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

import matplotlib.pyplot as plt
plt.style.use("ggplot")

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/breast_cancer_dataset.csv")
df2 = df

In [25]:
df2.sample(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
415,905686,B,11.890,21.17,76.39,433.8,0.09773,0.08120,0.02555,0.02179,0.2019,0.06290,0.2747,1.2030,1.930,19.53,0.009895,0.030530,0.01630,0.009276,0.02258,0.002272,13.050,27.21,85.09,522.9,0.14260,0.21870,0.1164,0.08263,0.3075,0.07351,NaN
175,872113,B,8.671,14.45,54.42,227.2,0.09138,0.04276,0.00000,0.00000,0.1722,0.06724,0.2204,0.7873,1.435,11.36,0.009172,0.008007,0.00000,0.000000,0.02711,0.003399,9.262,17.04,58.36,259.2,0.11620,0.07057,0.0000,0.00000,0.2592,0.07848,NaN
545,922576,B,13.620,23.23,87.19,573.2,0.09246,0.06747,0.02974,0.02443,0.1664,0.05801,0.3460,1.3360,2.066,31.24,0.005868,0.020990,0.02021,0.009064,0.02087,0.002583,15.350,29.09,97.58,729.8,0.12160,0.15170,0.1049,0.07174,0.2642,0.06953,NaN
157,8711216,B,16.840,19.46,108.40,880.2,0.07445,0.07223,0.05150,0.02771,0.1844,0.05268,0.4789,2.0600,3.479,46.61,0.003443,0.026610,0.03056,0.011100,0.01520,0.001519,18.220,28.07,120.30,1032.0,0.08774,0.17100,0.1882,0.08436,0.2527,0.05972,NaN
516,916799,M,18.310,20.58,120.80,1052.0,0.10680,0.12480,0.15690,0.09451,0.1860,0.05941,0.5449,0.9225,3.218,67.36,0.006176,0.018770,0.02913,0.010460,0.01559,0.002725,21.860,26.20,142.20,1493.0,0.14920,0.25360,0.3759,0.15100,0.3074,0.07863,NaN


The class distribution can be found by counting the diagnosis column. B for benign and M for malignant.

In [29]:
df['diagnosis'].value_counts

KeyError: ignored

Convert the class labels and split the data into training and test sets. train_test_split with stratify=True results in consistent class distribution between training and test sets:

In [26]:
# by default majority class(benign) will be negative
lb = LabelBinarizer()
df['diagnosis'] = lb.fit_transform(df['diagnosis'].values)
targets = df['diagnosis']

df.drop(['id', 'diagnosis', 'Unnamed: 32'], axis=1, inplace=True)

#train test split
X_train, X_test, y_train, y_test = train_test_split(df, targets, stratify=targets, random_state=42)

In [32]:
# show the distribution
print('y_train class distribution')
print(y_train.value_counts(normalize=True))

print('y_test class distribution')
print(y_test.value_counts(normalize=True))


y_train class distribution
0    0.626761
1    0.373239
Name: diagnosis, dtype: float64
y_test class distribution
0    0.629371
1    0.370629
Name: diagnosis, dtype: float64


Now that the data has been prepared, the classifier can be built.

## First strategy: Optimize for sensitivity using GridSearchCV with the scoring argument.

First build a generic classifier and setup a parameter grid; random forests have many tunable parameters, which make it suitable for GridSearchCV. The scorers dictionary can be used as the scoring argument in GridSearchCV. When multiple scores are passed, GridSearchCV.cv_results_ will return scoring metrics for each of the score types provided.

In [36]:
clf = RandomForestClassifier()

param_grid = {
    'min_samples_split': [3, 5 , 10],
    'n_estimators': [100, 300],
    'max_depth':[3, 5, 15, 25],
    'max_features': [3, 5, 10, 20]
}

scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}

The function below uses GridSearchCV to fit several classifiers according to the combinations of parameters in the param_grid. The scores from scorers are recorded and the best model (as scored by the refit argument) will be selected and "refit" to the full training data for downstream use. This also makes predictions on the held out X_test and prints the confusion matrix to show performance.
The point of the wrapper function is to quickly reuse the code to fit the best classifier according to the type of scoring metric chosen. First, try precision_score, which should limit the number of false positives. This isn't well-suited for the goal of maxium sensitivity, but allows us to quickly show the difference between a classifier optimized for precision_score and one optimized for recall_score.

In [41]:
def grid_search_wrapper(refit_score='precision_score'):
  """
  fits a GridSearchCV classifier using refit_score for optimisation
  prints classifier performance metrics
  """
  skf = StratifiedKFold(n_splits=10)
  grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score, cv=skf, return_train_score=True, n_jobs=-1)
  grid_search.fit(X_train.values, y_train.values)

  #make the predictions
  y_pred = grid_search.predict(X_test.values)

  print('Best params for {}'.format(refit_score))
  print(grid_search.best_params_)

  # confusion matrix on the test data
  print('\nConfusion matrix of Random Forest optimised for {} on the test data:'.format(refit_score))
  print(pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['pred_neg', 'pred_pos'], index=['neg','pos']))

  return grid_search

In [42]:
grid_search_clf = grid_search_wrapper(refit_score='precision_score')

Best params for precision_score
{'max_depth': 3, 'max_features': 20, 'min_samples_split': 10, 'n_estimators': 100}

Confusion matrix of Random Forest optimised for precision_score on the test data:
     pred_neg  pred_pos
neg        90         0
pos         6        47
